In [1]:
import numpy as np
import math
from scipy.stats import norm, gamma, poisson
import operator
import pandas as pd
import sys
from functools import reduce
import time

In [3]:
SeattleA = np.array([[15,12,4],[11,17,4],[0,21,4],[0,0,5]])
SeattleA

array([[15, 12,  4],
       [11, 17,  4],
       [ 0, 21,  4],
       [ 0,  0,  5]])

In [294]:
def process(DATA):
    hsA = np.sum(DATA,axis=0)
    isA = np.sum(DATA,axis=1)
    colA = np.array(range(1,len(hsA)+1))
    rowA = np.array(range(0,len(hsA)+1))
    
    xA = np.sum(isA*rowA) # Final size
    N = np.sum(hsA*colA) # Population size
    
    return {'hsA' : hsA, 'isA' : isA, 'colA' : colA, 'rowA' : rowA, 'xA' : xA, 'N' : N}

def PCOUP(Xdata, epss, k, run):
    """Partially coupled ABC algorithm to obtain run accepted values."""
     #initialize empty zero array
    output = np.zeros(((2*epss[1]+1)*run, 5), dtype = float)
    simcount=0
    count=0
    jj=0
    
    while jj<run:
        simcount+=1
        lambda_L = np.random.exponential(1,1) # Sample lambda_L
        J = House_COUP(Xdata,epss[1],lambda_L,k) # Run coupled simulations
        W = J[J[:,3]<J[:,4],:]        # W contains successful simulations (infect close to xA individuals)
        if W.size == 5:
            W = np.array(W).reshape((-1, 5))
        if W[:,0].size > 0:
            if min(W[:, 1])<=epss[0]:
                jj+=1
                for ii in range(W[:, 0].size):
                    if W[ii, 1]<=epss[0]:
                        count+=1
                        output[count-1,:]=np.r_[W[ii,1:4],lambda_L]
                        
    # Stores values from simulation - these include closeness of simulated epidemic 
    # to data, range of lambda_G values and lambda_L
    return {'OUTPUT':output[0:count,:], 'simcount':simcount}


def House_COUP(Xdata,epsil,lambda_L,k):
    """Partially coupled ABC algorithm for household epidemics
    lambda_L is drawn from the prior (or however)
    Code finds lambda_G values consistent with the data
    Input: Xdata - Epidemic data to compare simulations with.
    epsil - Max distance between simulated and observed final size for a simulation 
    to be accepted. (Tighter control on distance after simulations straightforward).
    lambda_L - local infection (household) rate
    k - Gamma(k,k) infectious period with k=0 a constant infectious period."""
    hsA = np.sum(Xdata, axis = 0) # hsA[i] - Number of households of size i
    isA = np.sum(Xdata, axis = 1) # isA[i] - Number of households with i-1 infectives
    colA = np.arange(1, hsA.shape[0]+1)
    rowA = np.arange(0, hsA.shape[0]+1)
    
    xA = np.sum(isA*rowA) # Final size
    HH = hsA.shape[0] # HH maximum household size
    ks = np.arange(1, HH+1)
    
    n = np.repeat(ks, hsA, axis=0)
    m = n.shape[0]# Number of households
    N = np.sum(n) # Population size
    NS = N # Number of susceptibles
    sev=0       # Running tally of severity (sum of infectious periods)
    threshold=0 # Running tally of (global) threshold required for the next infection
    
    ni = np.repeat(0, n.shape[0], axis = 0) # infectives (per household)
    ns = n.copy() # susceptibles (per household)
    
    OUT = np.zeros((HH+1, HH))
    OUT[0, :] = hsA # Epidemic data in the same form as Xdata
                   # Start with everybody susceptible
    
    DISS = np.zeros((2*epsil+1, 5)) # Matrix for collecting epidemics infecting within epsil of xA infectives.
    SEVI = np.zeros((N, 3)) # Matrix to keep track of number of infectives, severity and threshold.
    ys=0    # number of infectives
    count=0 # number of global infections taking place. First global infection is the introductory case. 
    
    while ys<=(xA+epsil):
        # Only need to consider the epidemic until xA+epsil infections occur.
        # We simulate successive global infections (should they occur) with associated
        # local (within household epidemics)
        # For the count+1 global infection to take place, we require that 
        # for k=1,2,..., count;  lambda_G * severity from first k infectives is larger
        # than the k^th threshold
        count+=1
        kk = np.random.choice(m, 1, p = ns/ns.sum(), replace = True)[0]
        OUT[ni[kk-1],n[kk-1]-1]=OUT[ni[kk-1],n[kk-1]-1]-1
        hou_epi=House_epi(ns[kk-1],k,lambda_L)# Simulate a household epidemic among the remaining susceptibles in the household
        
        ns[kk-1]-=hou_epi[0]
        ni[kk-1] = n[kk-1] - ns[kk-1]#update household kk data (susceptibles and infectives)
        
        OUT[ni[kk-1],n[kk-1]-1]+=1# Update the state of the population following the global 
        #infection and resulting household epidemic
        NS = ns.sum()
        threshold+=np.random.exponential(size = 1, scale = (N/NS))
        
        ys+=hou_epi[0]
        sev+=hou_epi[1]
        SEVI[count-1,:] = [ys,sev,threshold]
        # If the number infected is close to xA, we check what value of lambda_G 
        # would be needed for an epidemic of the desired size. 
        # Note that in many cases no value of lambda_G will result in an epidemic 
        # close to xA. 
        if abs(ys-xA)<=epsil:
            dist = np.sum(abs(OUT-Xdata))
            TT = SEVI[0:count, 2]/SEVI[0:count, 1] #ratio of threshold to severity
            Tlow = max(TT[0:count])
            Thi=TT[0:count].max()   #  Thi is the maximum lambda_G which leads to at most count global infections
            DISS[(ys-(xA-epsil)).astype('int'), :] = [1,dist,abs(ys-xA),Tlow,Thi]
            
    return DISS


def House_epi(n,k,lambda_L):
    
    i=0
    sev=0
    
    if n == 1:
        i = 1
        if k == 0:
            sev = 1
        if k > 0:
            sev = np.random.gamma(k, 1/k, 1)
    
    if n > 1:
        t = thresH(n)
        if k == 0:
            q = np.repeat(1.0,n)
        if k > 0:
            q = np.random.gamma(size=n, shape=k, scale=1/k)
        t = np.append(t, 2*lambda_L*np.sum(q))
        
        i = 0
        test = 0
        while test == 0:
            i += 1
            if t[i-1] > (lambda_L*np.sum(q[0:i])):
                test = 1 
                sev = np.sum(q[0:i])
                
    return np.array([i,sev])


In [295]:
DATA=SeattleA

# Main code for partially coupled ABC for household epidemics
tstartC = time.time()

# Process data
proc = process(DATA)
hsA = proc['hsA']
isA = proc['isA']
colA = proc['colA']
rowA = proc['rowA']
xA = proc['xA']
N = proc['N']

# Set precision thresholds (epss), infectious period (k) and
# number of iterations (run)
epss = np.array((8,1))
k = 0
run = 10

OUTP = PCOUP(DATA,epss,k,run)
OUT=OUTP['OUTPUT']
OUTP['simcount']

IndexError: index 177 is out of bounds for axis 0 with size 177

In [43]:
#
# Main code for partially coupled ABC for household epidemics
#
#

tstartC=proc.time()

#
# Set dataset: SeattleA, SeattleB, Tecumseh1, Tecumseh2
#

data(SeattleA)
DATA=SeattleA

#
# Process data
#

proc = process(DATA)
hsA = proc[[1]]; isA = proc[[2]]; colA = proc[[3]]; rowA = proc[[4]]; xA = proc[[5]]; N = proc[[6]]

#
# Set precision thresholds (epss), infectious period (k) and
# number of iterations (run)
#

epss=c(8,1)
k=0
run=1000

OUTP=PCOUP(DATA,epss,k,run)
OUT=OUTP$OUTPUT
OUTP$simcount

#
# Compute posterior means and standard deviations for partially coupled ABC
#

wei=0
moMG=rep(0,2)
moML=rep(0,2)

Count=length(OUT[,1])
# Number of stored values - note there might be one for each accepted simulation

for(i in 1:Count)
{
wei=wei+Mexp(0,1,0,OUT[i,3],OUT[i,4])
for(j in 1:2)
{
moMG[j]=moMG[j]+Mexp(j,1,0,OUT[i,3],OUT[i,4])
moML[j]=moML[j]+Mexp(0,1,0,OUT[i,3],OUT[i,4])*OUT[i,5]^j
}
}

meanG=moMG[1]/wei
sdG=sqrt(moMG[2]/wei-meanG^2)
meanL=moML[1]/wei
sdL=sqrt(moML[2]/wei-meanL^2)

#
# Computes transformed means and standard deviations of
# q_G = exp(-lambdaG * xA/N); q_L = exp(-lambdaL)
#

A1=1+xA/N
A2=1+2*xA/N

WEIq=exp(-OUT[,3])-exp(-OUT[,4])
moqG=sum(exp(-A1*OUT[,3])-exp(-A1*OUT[,4]))/A1
moqG[2]=sum(exp(-A2*OUT[,3])-exp(-A2*OUT[,4]))/A2

moqL=sum(WEIq*exp(-OUT[,5]))
moqL[2]=sum(WEIq*exp(-2*OUT[,5]))

meanqG=moqG[1]/wei
sdqG=sqrt(moqG[2]/wei-meanqG^2)
meanqL=moqL[1]/wei
sdqL=sqrt(moqL[2]/wei-meanqL^2)

#
# Summarise results
#

# Parameter means and sd
c(meanG,sdG,meanL,sdL)

# Transformed parameters means and sd (compare with Clancy and O'Neill (2008)
# and Neal (2012))
c(meanqG,sdqG,meanqL,sdqL)

# Time taken.

tendC=proc.time()
tendC-tstartC


SyntaxError: invalid syntax (<ipython-input-43-aa92c01c6d15>, line 32)